# Battle of the Neighbourhoods: Hamburg, Germany

# Table of contents
* [Introduction](#Introduction)
* [Data](#Data)
* [Methodology](#Methodology)
  - [Import and Read](#sub1)
  - [Functions for Radius and Coordinates](#sub2)
  - [Create new columns](#sub3)
  - [Libraries for Maps](#sub4)
  - [Map of Hamburg neighbourhoods](#sub5)
  - [Function for querying Foursquare](#sub6)
  - [Get venues](#sub7)
  - [Total number of venues](#sub8)
  - [One hot encoding](#sub9)
  - [Join to original table](#sub10)
  - [Sum and Mean](#sub11)
  - [Venues per population](#sub12)
  - [If a new venue is opened](#sub13)
* [Results](#Results)
  - [Grocery Stores](#rsub1)
  - [Supermarkets](#rsub2)
  - [Parks](#rsub3)
* [Discussion](#Discussion)
* [Conclusion](#Conclusion)

## Introduction

At about 1.9 million population, Hamburg is the second most-populated city in Germany. A former free trade city of the Hanseatic League of merchants, it remains a wealthy city-state to this day, housing many corporate headquarters and offices. Its wealth as measured by GDP per capita is second only to the likes of Frankfurt and Wolfsburg (with its Volkswagen headquarter and manufacturing plants). Hamburg should thus be of business interest to investors and business owners alike.

The goal is to utilize Foursquare location data along with other available data on the city of Hamburg to hopefully obtain some insights and identify business opportunities. In particular, we aim to identify in which neighbourhood(s) in Hamburg that one should open a new supermarket or grocery store.

## Data

Conveniently, the German wikipedia page 'Liste der Bezirke und Stadtteile Hamburgs' (list of districts and city parts of Hamburg) already lists all the city parts (i.e. neighbourhoods) along with their area, population, density, and even coordinates. These coordinates can be supplied to Foursquare application to obtain venues and business information, which will then be associated with each of these Hamburg neighbourhoods, providing us with data which can be readily processed and analyzed right away. 

## Methodology

Based on the data we have on hand, we decided that we have enough data to make recommendations on where to open a new grocery store, supermarket, and even a new park, namely because along with venues and coordinates, we also possess population data. For groceries, supermarkets, and parks, the proximity to one's residence should be key to the decision to patronize the venue. The same probably cannot be said of other types of shops such as restaurants, cafes, or bar, at least not to same extent.

### Import necessary libraries, read the table, print out data types, and assign column names <a name="sub1"></a>

In [3]:
import pandas as pd
import math
import numpy as np
pd.set_option('display.max_columns', None)

url='https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Stadtteile_Hamburgs'
df=pd.read_html(url, header=0, thousands='.', decimal=',')[1]

print(df.dtypes)
df.columns = ['Stadtteil','Ortsteile','Bezirk','Area','Population','Density','Coordinates','Map']
df.head(10)

Stadtteil                             object
Ortsteile                            float64
Bezirk                                object
Fläche(km²)                          float64
Einwohner                              int64
Bevölkerungsdichte(Einwohner/km²)      int64
Koordinaten                           object
Karte                                float64
dtype: object


,Stadtteil,Ortsteile,Bezirk,Area,Population,Density,Coordinates,Map
0,Hamburg-Altstadt,NaN,Hamburg-Mitte,NaN,2272,947,"53° 33′ 0″ N, 10° 0′ 0″ O",NaN
1,HafenCity,NaN,Hamburg-Mitte,NaN,4592,2087,"53° 32′ 28″ N, 10° 0′ 1″ O",NaN
2,Neustadt,NaN,Hamburg-Mitte,NaN,12920,5617,"53° 33′ 7″ N, 9° 59′ 8″ O",NaN
3,St. Pauli,NaN,Hamburg-Mitte,NaN,22436,8974,"53° 33′ 25″ N, 9° 57′ 50″ O",NaN
4,St. Georg,NaN,Hamburg-Mitte,NaN,11384,4743,"53° 33′ 18″ N, 10° 0′ 44″ O",NaN
5,Hammerbrook,NaN,Hamburg-Mitte,NaN,4323,1441,"53° 32′ 43″ N, 10° 1′ 50″ O",NaN
6,Borgfelde,NaN,Hamburg-Mitte,NaN,7696,9620,"53° 33′ 17″ N, 10° 2′ 4″ O",NaN
7,Hamm,NaN,Hamburg-Mitte,NaN,38773,9693,"53° 33′ 39″ N, 10° 3′ 28″ O",NaN
8,Horn,NaN,Hamburg-Mitte,NaN,38799,6576,"53° 33′ 14″ N, 10° 5′ 24″ O",NaN
9,Billstedt,NaN,Hamburg-Mitte,NaN,70355,4139,"53° 32′ 26″ N, 10° 6′ 4″ O",NaN


### Define necessary functions to extract coordinates and compute the radius of the area based on area size <a name="sub2"></a>

The getradius function works on the assumption that all Stadtteil (neighbourhoods) are circular-shaped, but to account for the reality that the neighbourhoods shapes do vary, we apply a factor of 0.8 to area size to reduce the computed radius. 

The radius (in metres) will be supplied to Foursquare application when querying venues such that it can be assumed that the venues all lie within the neighbourhood, and that there is little to no venue in the neighbourhood outside this search radius.

In [2]:
def coortolat(coor):
    coorlist=coor.replace('°', '').replace('′', '').replace('″', '').replace('N, ', '').replace('O', '').split()
    lat = float(coorlist[0]) + float(coorlist[1])/60 + float(coorlist[2])/3600
    return lat
def coortolong(coor):
    coorlist=coor.replace('°', '').replace('′', '').replace('″', '').replace('N, ', '').replace('O', '').split()
    long = float(coorlist[3]) + float(coorlist[4])/60 + float(coorlist[5])/3600
    return long
def getradius(area):
    return math.sqrt(area*0.8/math.pi)*1000

### The functions are then applied to create new columns <a name="sub3"></a>

In [3]:
df['Area']=df['Population']/df['Density']
df.drop(['Ortsteile','Map'], axis=1, inplace=True)

df['Latitude']=df['Coordinates'].apply(coortolat)
df['Longitude']=df['Coordinates'].apply(coortolong)
df.drop('Coordinates', axis=1, inplace=True)

df['Radius']=df['Area'].apply(getradius)

# drop the rows that density is 0
df=df[df.Density != 0].reset_index(drop=True)
df.head(10)

,Stadtteil,Bezirk,Area,Population,Density,Latitude,Longitude,Radius
0,Hamburg-Altstadt,Hamburg-Mitte,2.399155,2272,947,53.550000,10.000000,781.626421
1,HafenCity,Hamburg-Mitte,2.200287,4592,2087,53.541111,10.000278,748.530968
2,Neustadt,Hamburg-Mitte,2.300160,12920,5617,53.551944,9.985556,765.330642
3,St. Pauli,Hamburg-Mitte,2.500111,22436,8974,53.556944,9.963889,797.902343
4,St. Georg,Hamburg-Mitte,2.400169,11384,4743,53.555000,10.012222,781.791489
5,Hammerbrook,Hamburg-Mitte,3.000000,4323,1441,53.545278,10.030556,874.038744
6,Borgfelde,Hamburg-Mitte,0.800000,7696,9620,53.554722,10.034444,451.351667
7,Hamm,Hamburg-Mitte,4.000103,38773,9693,53.560833,10.057778,1009.266024
8,Horn,Hamburg-Mitte,5.900091,38799,6576,53.553889,10.090000,1225.742998
9,Billstedt,Hamburg-Mitte,16.998067,70355,4139,53.540556,10.101111,2080.510096


### Import additional libraries for querying coordinates and generating maps. <a name="sub4"></a>

In [4]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

## Package Plan ##

  environment location: D:\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                        anaconda::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1




conda-4.8.3          | 3.1 MB    |            |   0% 
conda-4.8.3          | 3.1 MB    |            |   1% 
conda-4.8.3          | 3.1 MB    | 6          |   7% 
conda-4.8.3          | 3.1 MB    | ##1        |  21% 
conda-4.8.3          | 3.1 MB    | ###2       |  3

### Generate a map to visualize the neighbourhood locations <a name="sub5"></a>

In [5]:
address = 'Hamburg'

geolocator = Nominatim(user_agent="hamburg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hamburg are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_hamburg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Stadtteil']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hamburg)  
    
map_hamburg

The geograpical coordinate of Hamburg are 53.5437641, 10.0099133.


In [6]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Define the function for querying venues from Foursquare <a name="sub6"></a>

Latitude, longitude, and radius of neighbourhoods are supplied to the request URL.

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng, rad in zip(names, latitudes, longitudes, radius):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            rad, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Call the defined function, setting a limit of returned venues to 500 per neighbourhood <a name="sub7"></a>

In [8]:
LIMIT = 500 # limit of number of venues returned by Foursquare API

hamburg_venues = getNearbyVenues(names=df['Stadtteil'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                   radius=df['Radius']
                                  )

Hamburg-Altstadt
HafenCity
Neustadt
St. Pauli
St. Georg
Hammerbrook
Borgfelde
Hamm
Horn
Billstedt
Billbrook
Rothenburgsort
Veddel
Wilhelmsburg
Kleiner Grasbrook
Steinwerder
Finkenwerder
Neuwerk
Altona-Altstadt
Sternschanze
Altona-Nord
Ottensen
Bahrenfeld
Groß Flottbek
Othmarschen
Lurup
Osdorf
Nienstedten
Blankenese
Iserbrook
Sülldorf
Rissen
Eimsbüttel
Rotherbaum
Harvestehude
Hoheluft-West
Lokstedt
Niendorf
Schnelsen
Eidelstedt
Stellingen
Hoheluft-Ost
Eppendorf
Groß Borstel
Alsterdorf
Winterhude
Uhlenhorst
Hohenfelde
Barmbek-Süd
Dulsberg
Barmbek-Nord
Ohlsdorf
Fuhlsbüttel
Langenhorn
Eilbek
Wandsbek
Marienthal
Jenfeld
Tonndorf
Farmsen-Berne
Bramfeld
Steilshoop
Wellingsbüttel
Sasel
Poppenbüttel
Hummelsbüttel
Lemsahl-Mellingstedt
Duvenstedt
Wohldorf-Ohlstedt
Bergstedt
Volksdorf
Rahlstedt
Lohbrügge
Bergedorf
Curslack
Altengamme
Neuengamme
Kirchwerder
Ochsenwerder
Reitbrook
Allermöhe
Billwerder
Moorfleet
Tatenberg
Spadenland
Neuallermöhe
Harburg
Neuland
Gut Moor
Wilstorf
Rönneburg
Langenbek
S

### Show the total number of venues and total venues for each neighbourhood <a name="sub8"></a>

In [9]:
print(hamburg_venues.shape)
print(hamburg_venues.columns)
hamburg_venues.groupby('Neighborhood').count()["Venue"].to_frame().head(10)

(3143, 7)
Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')


,Venue
Neighborhood,
Allermöhe,8
Alsterdorf,35
Altengamme,6
Altona-Altstadt,100
Altona-Nord,43
Bahrenfeld,31
Barmbek-Nord,39
Barmbek-Süd,51
Bergedorf,42


### Use one hot encoding to transform the venue categories into columns <a name="sub9"></a>

In [10]:
# one hot encoding
hamburg_onehot = pd.get_dummies(hamburg_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hamburg_onehot['Neighbourhood'] = hamburg_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hamburg_onehot.columns[-1]] + list(hamburg_onehot.columns[:-1])
hamburg_onehot = hamburg_onehot[fixed_columns]

hamburg_grouped = hamburg_onehot.groupby('Neighbourhood').sum().reset_index()
print(hamburg_grouped.shape)
hamburg_grouped.head()

(101, 313)


,Neighbourhood,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cruise,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Laundromat,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nature Preserve,Neighborhood,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Pakistani Restaurant,Palace,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Swabian Restaurant,Szechuan Restaurant,Tapas Restaurant,Taverna,Tea Room,T

### Join the new venue table to the original neighbourhood and population table <a name="sub10"></a>

In [11]:
hamburgjoin = df.join(hamburg_grouped.set_index('Neighbourhood'),on='Stadtteil',how='right').reset_index(drop=True)
print(hamburgjoin.shape)
hamburgjoin.tail()

(101, 320)


,Stadtteil,Bezirk,Area,Population,Density,Latitude,Longitude,Radius,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cruise,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Laundromat,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nature Preserve,Neighborhood,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Pakistani Restaurant,Palace,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Swabian Restaurant,Sz

### Add 'sum' and 'mean' rows to the table, just for perspective <a name="sub11"></a>

In [12]:
hamburgjoinsum = hamburgjoin.sum().to_frame().transpose()
hamburgjoinsum[['Stadtteil','Bezirk']]='All'
hamburgjoinsum[['Latitude','Longitude','Radius']]= np.NaN
hamburgjoinsum['Density'] = hamburgjoinsum['Population']/hamburgjoinsum['Area']
hamburgjoinsum

hamburgjoinmean = hamburgjoin.mean().to_frame().transpose()
#hamburgjoinmean[['Stadtteil','Bezirk']]='Mean'
hamburgjoinmean[['Latitude','Longitude','Radius']]= np.NaN
hamburgjoinmean['Density'] = hamburgjoinsum['Population']/hamburgjoinsum['Area']
type(pd.DataFrame(['Mean']))
hamburgjoinmean.insert(0,'Bezirk',pd.DataFrame(['Mean']))
hamburgjoinmean.insert(0,'Stadtteil',pd.DataFrame(['Mean']))
hamburgjoinmean


hamburgjoin = hamburgjoin.append(hamburgjoinsum).append(hamburgjoinmean).reset_index(drop=True)
hamburgjoin.tail()

,Stadtteil,Bezirk,Area,Population,Density,Latitude,Longitude,Radius,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cruise,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Laundromat,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nature Preserve,Neighborhood,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Pakistani Restaurant,Palace,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Swabian Restaurant,Sz

### Convert the count of venues for each neighbourhood to 'venues per 1 million population' <a name="sub12"></a>

In [13]:
for i in range(8,hamburgjoin.shape[1]):
    hamburgjoin.iloc[:,i] = hamburgjoin.iloc[:,i]*1000000/hamburgjoin.iloc[:,3]
hamburgjoin.head()

,Stadtteil,Bezirk,Area,Population,Density,Latitude,Longitude,Radius,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cruise,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Laundromat,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nature Preserve,Neighborhood,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Pakistani Restaurant,Palace,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Swabian Restaurant,Sz

### Extract only the required columns <a name="sub13"></a> 
which are 'neighbourhood', 'population', 'grocery store', 'park', and 'supermarket'

### Add 1 venue into each of the extracted categories <a name="sub14"></a>
so we now have the 'venues per 1 million population, after someone opens a new venue'

The reasoning is that there are many lowly-populated neighbourhoods that, while lacking in venues, don't actually need one since the population is so small.

In [14]:
hamburghomeplus=hamburgjoin.loc[0:100,['Stadtteil','Population','Grocery Store','Park','Supermarket']]
for i in range(2,5):
    hamburghomeplus.iloc[:,i] = hamburghomeplus.iloc[:,i] + 1000000/hamburghomeplus.iloc[:,1]
hamburghomeplus.head(10)

,Stadtteil,Population,Grocery Store,Park,Supermarket
0,Hamburg-Altstadt,2272,880.282,440.141,440.141
1,HafenCity,4592,217.77,435.54,435.54
2,Neustadt,12920,77.3994,77.3994,77.3994
3,St. Pauli,22436,44.5712,89.1424,89.1424
4,St. Georg,11384,263.528,175.685,175.685
5,Hammerbrook,4323,231.321,231.321,231.321
6,Borgfelde,7696,129.938,259.875,129.938
7,Hamm,38773,25.7911,77.3734,257.911
8,Horn,38799,51.5477,51.5477,128.869
9,Billstedt,70355,28.4273,14.2136,99.4954


## Results

### Sort the new 'venues per million population' based on grocery stores <a name="rsub1"></a>

In [15]:
hamburghomeplus.sort_values('Grocery Store').head(10)

,Stadtteil,Population,Grocery Store,Park,Supermarket
71,Rahlstedt,91740,10.9004,10.9004,109.004
45,Winterhude,55900,17.8891,71.5564,89.4454
13,Wilhelmsburg,54068,18.4952,92.4761,73.9809
53,Langenhorn,45666,21.8981,43.7963,87.5925
50,Barmbek-Nord,41886,23.8743,23.8743,95.4973
37,Niendorf,40717,24.5598,49.1195,122.799
72,Lohbrügge,39809,25.1199,25.1199,125.6
7,Hamm,38773,25.7911,77.3734,257.911
55,Wandsbek,36149,27.6633,82.9898,138.316
48,Barmbek-Süd,35438,28.2183,28.2183,141.091


### Then sort by supermarkets <a name="rsub2"></a>

In [16]:
hamburghomeplus.sort_values('Supermarket').head(10)

,Stadtteil,Population,Grocery Store,Park,Supermarket
54,Eilbek,22233,44.9782,89.9564,44.9782
32,Eimsbüttel,58196,34.3666,68.7332,51.5499
21,Ottensen,35585,56.2035,84.3052,56.2035
18,Altona-Altstadt,29305,68.2477,68.2477,68.2477
13,Wilhelmsburg,54068,18.4952,92.4761,73.9809
56,Marienthal,13382,74.7272,149.454,74.7272
2,Neustadt,12920,77.3994,77.3994,77.3994
63,Sasel,23778,42.0557,84.1114,84.1114
59,Farmsen-Berne,34689,28.8276,28.8276,86.4827
53,Langenhorn,45666,21.8981,43.7963,87.5925


### Then sort by parks <a name="rsub3"></a>

In [17]:
hamburghomeplus.sort_values('Park').head(10)

,Stadtteil,Population,Grocery Store,Park,Supermarket
71,Rahlstedt,91740,10.9004,10.9004,109.004
9,Billstedt,70355,28.4273,14.2136,99.4954
60,Bramfeld,52498,38.0967,19.0483,190.483
50,Barmbek-Nord,41886,23.8743,23.8743,95.4973
72,Lohbrügge,39809,25.1199,25.1199,125.6
25,Lurup,36695,54.5033,27.2517,136.258
48,Barmbek-Süd,35438,28.2183,28.2183,141.091
59,Farmsen-Berne,34689,28.8276,28.8276,86.4827
39,Eidelstedt,33421,29.9213,29.9213,179.528
97,Neugraben-Fischbek,30690,32.5839,32.5839,130.336


There are the neighbourhoods that most need a new grocery store, a new supermarket, and a new park, respectively.

## Discussion

We identified the neighbourhoods in which one would most benefit from opening a new grocery store, supermarket, and park, with a rather simple methodology requiring no advanced machine learning or clustering techniques. It should nevertheless still be convincing as we were able to show the number of venues per population ratios of each neighbourhood which then tells us which neighbourhoods could benefit from an additional venue. Some of the recommended neighbourhoods such as Eimsbüttel, Altona, Winterhude, and Neustadt are fairly central locations and should be the safer choices to open a new venue than other recommended neighbourhoods which are further away and more loosely populated.

If one were to look closely, though, one would notice some peculiarities in the number of venues that Foursquare returned. For example, Foursquare data seems to suggest that there is only 1 ATM in the whole city of Hamburg. Kebab joints, coming in at 2, should not be the case as well as one can almost literally find one or more around every neighbourhood. This does suggest that there is a problem with Foursquare venue data. If this data is ever updated correctly, though, then we can definitely rely on our results. All this being said, a total of 228 supermarkets sounds like a reasonable number, so the results on supermarket is the most reliable of our three.

## Conclusion

The purpose of this project is to analyze the neighbourhoods of Hamburg in order to come up with good options for investing in a new grocery, supermarket, or park. This however is by no means a definite revelation and there should be other aspects and parameters at work in determining if and when a business is to be successful. Nonetheless it should have been of some use to narrow down the choices that an investor or business operator could choose in his/her endeavour at a new business and some of the recommended options are as mentioned fairly central so could be solid options in one's next investment.